In [ ]:
import pandas as pd

def process_girs_data(file_path):
    # Load required sheets
    conso_data = pd.read_excel(file_path, sheet_name='Conso Data')
    girs_data = pd.read_excel(file_path, sheet_name="GIRS Extract_20 Mar'25")
    tier1_data = pd.read_excel(file_path, sheet_name='TIER1_5 Mar'25')
    ttrl_data = pd.read_excel(file_path, sheet_name='TTRL_3 Mar'25')
    pdmr_data = pd.read_excel(file_path, sheet_name='PDMR_1Jan'25', header=None)  # PDMR has one column
    
    # Standardizing column names
    conso_data.rename(columns={'Resource name': 'Insider Name'}, inplace=True)
    girs_data.rename(columns={'Employee ID': 'PSID', 'Employee Email': 'Email'}, inplace=True)
    tier1_data.rename(columns={'Employee ID': 'PSID', 'Employee Email': 'Email'}, inplace=True)
    ttrl_data.rename(columns={'Employee ID': 'PSID', 'Employee Email': 'Email'}, inplace=True)
    
    # Convert 'PDMR' column to lowercase for matching
    pdmr_data[0] = pdmr_data[0].str.lower()
    
    # Remove leading zeros from PSID
    conso_data['PSID'] = conso_data['PSID'].astype(str).str.lstrip('0')
    girs_data['PSID'] = girs_data['PSID'].astype(str).str.lstrip('0')
    tier1_data['PSID'] = tier1_data['PSID'].astype(str).str.lstrip('0')
    ttrl_data['PSID'] = ttrl_data['PSID'].astype(str).str.lstrip('0')
    
    # Capitalize Insider Names
    conso_data['Insider Name'] = conso_data['Insider Name'].str.upper()
    girs_data['Insider Name'] = girs_data['Insider Name'].str.upper()
    tier1_data['Insider Name'] = tier1_data['Insider Name'].str.upper()
    ttrl_data['Insider Name'] = ttrl_data['Insider Name'].str.upper()
    
    # Merge GIRS Current Status into Conso Data
    merged_data = conso_data.merge(girs_data[['PSID', 'Insider Name', 'Email', 'Role']], 
                                   on=['PSID', 'Insider Name'], how='left')
    merged_data.rename(columns={'Role': 'Original Status in GIRS'}, inplace=True)
    
    # Identify if PSID is present in Tier1, TTRL
    merged_data['Tier1 Flag'] = merged_data['PSID'].isin(tier1_data['PSID']).map({True: 'Yes', False: 'No'})
    merged_data['TTRL Flag'] = merged_data['PSID'].isin(ttrl_data['PSID']).map({True: 'Yes', False: 'No'})
    
    # Identify if Insider Name matches with PDMR data
    merged_data['PDMR Flag'] = merged_data['Insider Name'].str.lower().isin(pdmr_data[0]).map({True: 'Yes', False: 'No'})
    
    # Determine Updated Status Request based on rules
    def determine_updated_status(row):
        if row['Status'] == 'ADD':
            return 'Non-Core' if row['Original Status in GIRS'] == 'Inactive Insider' else 'ADD'
        elif row['Status'] == 'DEL':
            return 'Inactive Insider' if row['Original Status in GIRS'] == 'Non-Core' else 'DEL'
        elif row['Status'] == 'Cur':
            return row['Original Status in GIRS']  # Keep as is
        return row['Status']
    
    merged_data['Updated Status Request'] = merged_data.apply(determine_updated_status, axis=1)
    
    # Identify Request Type (New/Updated Request)
    merged_data['Request Type'] = merged_data['Original Status in GIRS'].apply(
        lambda x: 'New Request' if pd.isna(x) else 'Updated Request'
    )
    
    # Generate Comments Column
    def generate_comment(row):
        comments = []
        if row['Status'] == 'ADD' and row['Updated Status Request'] == 'ADD':
            comments.append('Fresh Addition')
        if row['Status'] == 'DEL' and row['Updated Status Request'] == 'Inactive Insider':
            comments.append('Converted to Inactive Insider')
        if row['Status'] == 'ADD' and row['Updated Status Request'] == 'Non-Core':
            comments.append('Reactivated to Non-Core')
        if row['Tier1 Flag'] == 'Yes':
            comments.append('Present in Tier1')
        if row['TTRL Flag'] == 'Yes':
            comments.append('Present in TTRL')
        if row['PDMR Flag'] == 'Yes':
            comments.append('Listed in PDMR')
        return '; '.join(comments)
    
    merged_data['Comment'] = merged_data.apply(generate_comment, axis=1)
    
    # Select final required columns
    final_columns = ['PSID', 'Insider Name', 'Email', 'Original Status in GIRS',
                     'Updated Status Request', 'Request Type', 'Comment']
    final_data = merged_data[final_columns]
    
    # Save output to a new Excel file
    output_file = 'Final_Consolidated_Data.xlsx'
    final_data.to_excel(output_file, index=False)
    print(f'Final consolidated data saved as {output_file}')
    
    return final_data

# Example usage:
file_path = 'GIRS_Processing.xlsx'  # Update this with your actual file path
final_output = process_girs_data(file_path)
